### Create new price list in OCS

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'

file_new_tariff_s1 = 'Partner_1_6_2019.csv'
file_new_tariff_s2 = 'P4_20190801.csv'
file_new_tariff_s4 = 'MB_1_1_2019_for_tariffs.csv'
file_allowed_list = 'DMI.dmi_allowed_list_export_Tue_Jul_09_2019.csv'

eur_usd = 1.2
increment = 2

roaming_plan_id_s1 = 267 # Partner
roaming_plan_id_s2 = 268 # P4
roaming_plan_id_s4 = 329 # MB

In [2]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,MCCMNC FROM
(
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID
,np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
,ROW_NUMBER() OVER(PARTITION BY c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE ORDER BY t.TADIG_CODE DESC) AS ROW_ID
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
) t
WHERE ROW_ID=1
'''
df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()
display(df_tadig.head())

sql_srt=\
'''
SELECT * FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329)
AND r.end_date IS NULL
'''

df_roaming_plan_rules = pd.read_sql_query(sql_srt, cnxn)

df_roaming_plan_rules_s1 = df_roaming_plan_rules[df_roaming_plan_rules.ROAMING_PLAN_ID == roaming_plan_id_s1]

df_roaming_plan_rules_s2 = df_roaming_plan_rules[df_roaming_plan_rules.ROAMING_PLAN_ID == roaming_plan_id_s2]

df_roaming_plan_rules_s4 = df_roaming_plan_rules[df_roaming_plan_rules.ROAMING_PLAN_ID == roaming_plan_id_s4]

print('Partner OCS tariffs')
display(df_roaming_plan_rules_s1.head())
print('P4 OCS tariffs')
display(df_roaming_plan_rules_s2.head())
print('MB OCS tariffs')
display(df_roaming_plan_rules_s4.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


Partner OCS tariffs


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN
1,5567.0,267.0,59.0,1.0,0.990086,0.990086,0.249622,0.0,10.238187,1.0,2017-11-01,None,False,None
2,5568.0,267.0,95.0,1.0,2.189060,2.189060,0.450132,0.0,14.333252,1.0,2017-11-01,None,False,None
3,5570.0,267.0,108.0,1.0,0.660066,0.000000,0.299730,0.0,9.997989,1.0,2017-11-01,None,False,None
4,5571.0,267.0,113.0,1.0,2.367968,0.000000,0.494058,0.0,15.786522,1.0,2017-11-01,None,False,None
5,5600.0,267.0,325.0,1.0,0.599000,0.000000,0.199000,0.0,6.998000,1.0,2018-02-01,None,False,None


P4 OCS tariffs


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN
0,4391.0,268.0,171.0,2.0,2.6520,0.000,0.282,0.0,1.5600,1.0,2017-12-01,None,False,None
49,22113.0,268.0,465.0,2.0,3.1800,0.000,0.768,0.0,14.2800,1.0,2019-04-10,None,False,False
50,22114.0,268.0,1181.0,2.0,2.1600,0.000,0.600,0.0,18.0000,1.0,2019-04-10,None,False,False
51,22115.0,268.0,1153.0,2.0,5.2800,5.280,0.840,0.0,9.0000,1.0,2019-04-10,None,False,False
52,22116.0,268.0,605.0,2.0,1.9056,0.618,0.258,0.0,3.0912,1.0,2019-04-10,None,False,False


MB OCS tariffs


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN
253,22852.0,329.0,392.0,4.0,0.013,0.013,0.13,0.0,0.003150,1.0,2019-04-19,None,True,False
254,22854.0,329.0,567.0,4.0,0.130,0.130,0.13,0.0,0.005040,1.0,2019-04-19,None,True,False
255,22855.0,329.0,199.0,4.0,0.260,0.260,0.26,0.0,0.006301,1.0,2019-04-19,None,True,False
256,22856.0,329.0,516.0,4.0,0.260,0.260,0.26,0.0,0.003150,1.0,2019-04-19,None,True,False
257,22857.0,329.0,499.0,4.0,0.260,0.260,0.26,0.0,0.006300,1.0,2019-04-19,None,True,False


In [4]:
# Загружаем новые тарифы Partner, P4, MB

df_partner = pd.read_csv(join(load_folder, file_new_tariff_s1),sep=';')
print('Тарифы Partner:')
display(df_partner.head(3))

df_p4 = pd.read_csv(join(load_folder, file_new_tariff_s2),sep=',')
print('Тарифы P4:')
display(df_p4.head(3))

df_mb = pd.read_csv(join(load_folder, file_new_tariff_s4),sep=';')
print('Тарифы MB:')
display(df_mb.head(3))

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602,Vodafone AL,Albania,VF,0.03,0,0.0062,0.003,1,1
1,CZECM,23003,Vodafone CZ,Czech Republic,VF,0.03,0,0.0062,0.003,1,1
2,DEUD2,26202,Vodafone DE,Germany,VF,0.03,0,0.0062,0.003,1,1


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE
0,Albania,TELEKOM ALBANIA SH.A,ALBAM,0.449,NaN,0.052,0.040,1,1 kB,yes,yes
1,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,60,1 kB,yes,yes
2,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,60,1 kB,yes,no


Тарифы MB:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,HK,3HK,HKGH3,0.013,0.013,0.13,0.00315,60,1
1,HK,3HK,HKGHT,0.013,0.013,0.13,0.00315,60,1
2,Macau,3 Macau,MACHT,0.130,0.130,0.13,0.00504,60,1


In [5]:
# Загружаем Allowed list с необходимыми изменениями

df_al=pd.read_csv(join(load_folder, file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

print('Allowed list:')
display(df_al.head(3))

Allowed list:


,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,HRVT2,225.0
1,S2,MNEMT,622.0
2,S1,MKDCC,569.0


In [6]:
# Объединяем тарифы с TADIG и Allowed list

# Результирующий файл с тарифами Partner и P4 и укащаеием принадлежности в Combined

file_partner_p4_compare = 'Partner_P4_Compare_1_1_2019.csv'

# Неизвестные TADIG в тарифах Partner с добавлением network_id
df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

# Неизвестные TADIG в тарифах P4 с добавлением network_id
df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

# Неизвестные TADIG в тарифах MB с добавлением network_id
df_mb_net_id = pd.merge(df_mb,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах MB с добавлением network_id:')
display(df_mb_net_id[df_mb_net_id.TADIG_CODE.isnull()])

# Тарифы Partner и P4
df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()

# Тарифы Partner и P4 с TADIG
df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')

# Тарифы Partner и P4 с привязкой к Allowed list
df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')

# Результирующий файл
# Тарифы Partner и P4 с привязкой к Allowed list

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2',
          'SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()

#df_s1_s2_sponsor.to_csv(join(load_folder,file_partner_p4_compare),index=False)

df_s1_s2_sponsor.head(3)

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
144,Mozambique,MOVITEL S.A,MOZVG,0.7,NaN,0.07,0.042,60,10 kB,yes,no,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах MB с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.03,0.0,0.0062,0.003,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Czech Republic,Vodafone Czech Republic a s,CZECM,0.03,0.0,0.0062,0.003,1.0,1.0,0.035,NaN,0.013,0.0060,1,1 kB,S2,CZECM,238.0
2,Germany,Vodafone GmbH,DEUD2,0.03,0.0,0.0062,0.003,1.0,1.0,0.045,NaN,0.013,0.0063,1,1 kB,S2,DEUD2,338.0


In [7]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df[df.values > 1]
display(s2)

df = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)

df = df_mb_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s4 = df[df.values > 1]
display(s4)

,MOC,MTC,SMS,MB
NETWORK_ID,,,,
412.0,2,0,1,1


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [8]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Partner

# Файлы:
# Переход disc -> Not disc для отправки спонсору для запроса тарифов
# Для генерации таблицы в БД

file_s1_disc_to_not_disc = 'df_s1_disc_to_not_disc.csv'
file_s1_to_db = 'df_s1_insert_267.csv'
file_s1_to_db_100 = 'df_s1_insert_267_100.csv'

# Объединение новых тарифов со старыми, выгруженными из базы данных
df_temp = pd.merge(df_s1_s2_sponsor.loc[df_s1_s2_sponsor.MB_S1.notnull(),
                                        ['NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1']],
         df_roaming_plan_rules_s1[['ROAMING_PLAN_RULE_ID','NETWORK_ID','is_discounted',
                                   'MO_CALL_RATE', 'MT_CALL_RATE', 'MO_SMS_RATE', 'MT_SMS_RATE','DATA_RATE']],
         how='outer',on='NETWORK_ID').drop_duplicates()

# New
df = df_temp[df_temp.is_discounted.isnull()].copy()
df['com'] = 'new'
df_s1_to_db = df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']].copy()

# Not disc -> disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S1.notnull())].copy()
df['com'] = 'Not disc -> disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# Not disc -> Not disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S1.isnull())].copy()
df.drop(['MOC_S1','MTC_S1','SMS_S1','MB_S1'],axis=1,inplace=True)
df.rename({'MO_CALL_RATE':'MOC_S1','MT_CALL_RATE':'MTC_S1','MO_SMS_RATE':'SMS_S1','DATA_RATE':'MB_S1'},axis=1,inplace=True)
df['com'] = 'Not disc -> Not disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# disc -> Not disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S1.isnull())].copy()
df['com'] = 'disc -> Not disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])
df = pd.merge(df,df_tadig,how='left',on='NETWORK_ID')\
[['TADIG_CODE','ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']]
df.to_csv(join(load_folder,file_s1_disc_to_not_disc),index=False)
print('disc -> Not disc:')
display(df)

# disc -> disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S1.notnull())].copy()
df['com'] = 'disc -> disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# Добавляем поля для удобства выгрузки
#!!! Поменять 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

#df_s1_to_db['ROAMING_PLAN_RULE_ID']=''
df_s1_to_db['ROAMING_PLAN_ID']=''
df_s1_to_db['SPONSOR_ID']=1
df_s1_to_db['MT_SMS_RATE']=0
df_s1_to_db['CURRENCY_ID']=1 #1:USD, 2:EUR
df_s1_to_db['START_DATE']='2019-01-01 00:00:00'
df_s1_to_db['END_DATE']='NULL'

df_s1_to_db = pd.merge(df_s1_to_db,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',on='NETWORK_ID')

df_s1_to_db_100 = df_s1_to_db.copy()
df_s1_to_db_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] = df_s1_to_db_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']]*2

df_s1_to_db[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db),index=False)

df_s1_to_db_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db_100),index=False)

disc -> Not disc:


,TADIG_CODE,ROAMING_PLAN_RULE_ID,NETWORK_ID,MOC_S1,MTC_S1,SMS_S1,MB_S1,is_discounted,com
0,INDIB,22947.0,419.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
1,DEUE1,22975.0,328.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
2,ARM05,19061.0,33.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
3,SGPST,19063.0,820.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
4,IRLME,19064.0,461.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
5,ITASI,19065.0,483.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
6,KORSK,19066.0,519.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
7,AIACW,19068.0,20.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
8,ATGCW,19069.0,23.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
9,ARG01,19070.0,27.0,NaN,NaN,NaN,NaN,True,disc -> Not disc


In [9]:
# Подготовка файла тарифов для загрузки в базу данных OCS для P4

# Файлы:
# Переход disc -> Not disc для отправки спонсору для запроса тарифов
# Для генерации таблицы в БД

file_s2_disc_to_not_disc = 'df_s2_disc_to_not_disc.csv'
file_s2_to_db = 'df_s2_inser_268.csv'
file_s2_to_db_100 = 'df_s2_insert_100.csv'


df_temp = pd.merge(df_s1_s2_sponsor.loc[df_s1_s2_sponsor.MB_S2.notnull(),
                                        ['NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2']],
         df_roaming_plan_rules_s2[['ROAMING_PLAN_RULE_ID','NETWORK_ID','is_discounted',
                                   'MO_CALL_RATE', 'MT_CALL_RATE', 'MO_SMS_RATE', 'MT_SMS_RATE','DATA_RATE']],
         how='outer',on='NETWORK_ID').drop_duplicates()

# New
df = df_temp[df_temp.is_discounted.isnull()].copy()
df['com'] = 'new'
df_s2_to_db = df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']].copy()

# Not disc -> disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S2.notnull())].copy()
df['com'] = 'Not disc -> disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

# Not disc -> Not disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S2.isnull())].copy()
df.drop(['MOC_S2','MTC_S2','SMS_S2','MB_S2'],axis=1,inplace=True)
df.rename({'MO_CALL_RATE':'MOC_S2','MT_CALL_RATE':'MTC_S2','MO_SMS_RATE':'SMS_S2','DATA_RATE':'MB_S2'},axis=1,inplace=True)
df['com'] = 'Not disc -> Not disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

# disc -> Not disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S2.isnull())].copy()
df['com'] = 'disc -> Not disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])
df = pd.merge(df,df_tadig,how='left',on='NETWORK_ID')\
[['TADIG_CODE','ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']]
df.to_csv(join(load_folder,file_s2_disc_to_not_disc),index=False)
display(df.head())

# disc -> disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S2.notnull())].copy()
df['com'] = 'disc -> disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

#df_s2_to_db['ROAMING_PLAN_RULE_ID']=''
df_s2_to_db['ROAMING_PLAN_ID']='268'
df_s2_to_db['SPONSOR_ID']=2
df_s2_to_db['MT_SMS_RATE']=0
df_s2_to_db['CURRENCY_ID']=2 #1:USD, 2:EUR
df_s2_to_db['START_DATE']='2019-01-01 00:00:00'
df_s2_to_db['END_DATE']='NULL'

df_s2_to_db = pd.merge(df_s2_to_db,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',on='NETWORK_ID')

df_s2_to_db_100 = df_s2_to_db.copy()
df_s2_to_db_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = df_s2_to_db_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']]*2

df_s2_to_db_100.loc[df_s2_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = \
df_s2_to_db_100.loc[df_s2_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC_S2','MTC_S2','SMS_S2','MB_S2']]*1.25

df_s2_to_db[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s2_to_db),index=False)

df_s2_to_db_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s2_to_db_100),index=False)

,TADIG_CODE,ROAMING_PLAN_RULE_ID,NETWORK_ID,MOC_S2,MTC_S2,SMS_S2,MB_S2,is_discounted,com
0,AIACW,22196.0,20.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
1,ATGCW,22197.0,23.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
2,BRBCW,22199.0,70.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
3,VGBCW,22200.0,122.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
4,CYMCW,22202.0,174.0,NaN,NaN,NaN,NaN,True,disc -> Not disc


In [15]:
df_mb_net_id.head(3)

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,HK,3HK,HKGH3,0.013,0.013,0.13,0.00315,60,1,Hong Kong (China),H3G/Hutchinson,HKGH3,392.0
1,HK,3HK,HKGHT,0.013,0.013,0.13,0.00315,60,1,Hong Kong (China),H3G/Hutchinson,HKGHT,392.0
2,Macau,3 Macau,MACHT,0.130,0.130,0.13,0.00504,60,1,Macao China,Hutchison Telephone Macau Company Limited,MACHT,567.0


In [9]:
# Подготовка файла тарифов для загрузки в базу данных OCS для MB

# Файлы:
# Для генерации таблицы в БД

file_s4_to_db = 'df_s4_inser_329.csv'
file_s4_to_db_100 = 'df_s4_insert_100.csv'

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df_s4_to_db = df_mb_net_id.copy()

df_s4_to_db['ROAMING_PLAN_RULE_ID']=''
df_s4_to_db['ROAMING_PLAN_ID'] = roaming_plan_id_s4
df_s4_to_db['SPONSOR_ID'] = 4
df_s4_to_db['MT_SMS_RATE'] = 0
df_s4_to_db['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_s4_to_db['START_DATE']='2019-01-01 00:00:00'
df_s4_to_db['END_DATE']='NULL'
df_s4_to_db['is_discounted']= 1

df_s4_to_db_100 = df_s4_to_db.copy()
df_s4_to_db_100[['MOC','MTC','SMS','MB']] = df_s4_to_db_100[['MOC','MTC','SMS','MB']]*2

df_s4_to_db_100.loc[df_s4_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','MB']] = \
df_s4_to_db_100.loc[df_s4_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','MB']]*1.25

df_s4_to_db[[
'TADIG_CODE',
#'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',]].drop_duplicates().to_csv(join(load_folder,file_s4_to_db),index=False)

df_s4_to_db_100[[
'TADIG_CODE',
#'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',]].drop_duplicates().to_csv(join(load_folder,file_s4_to_db_100),index=False)

In [10]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Real Combined Price Plan

# Файлы:
file_combined_100 = 'df_combined_100.csv'

df1 = df_s1_s2_sponsor[df_s1_s2_sponsor.SPONSOR=='S1']\
[['TADIG_CODE','MOC_S1','MTC_S1','SMS_S1','MB_S1','SPONSOR','NETWORK_ID','TADIG_CODE_AL']].copy()

df1.rename({'TADIG_CODE':'TADIG','MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA'},axis=1,inplace=True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df1['ROAMING_PLAN_RULE_ID']=''
df1['ROAMING_PLAN_ID']=''
df1['SPONSOR_ID']=1
df1['MT_SMS_RATE']=0
df1['CURRENCY_ID']=1 #1:USD, 2:EUR
df1['START_DATE']='2019-01-01 00:00:00'
df1['END_DATE']='NULL'
df1['is_discounted']=1

df2 = df_s1_s2_sponsor[df_s1_s2_sponsor.SPONSOR=='S2']\
[['TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR','NETWORK_ID','TADIG_CODE_AL']].copy()

df2.rename({'TADIG_CODE':'TADIG','MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA'},axis=1,inplace=True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df2['ROAMING_PLAN_RULE_ID']=''
df2['ROAMING_PLAN_ID']=''
df2['SPONSOR_ID']=2
df2['MT_SMS_RATE']=0
df2['CURRENCY_ID']=2 #1:USD, 2:EUR
df2['START_DATE']='2019-01-01 00:00:00'
df2['END_DATE']='NULL'
df2['is_discounted']=1

df_combined = pd.concat([df1, df2],axis=0).drop_duplicates()
df_combined.MTC.fillna(0,inplace=True)
df_combined.dropna(inplace=True)


df_combined_100 = df_combined.copy()
df_combined_100[['MOC','MTC','SMS','DATA']] = df_combined_100[['MOC','MTC','SMS','DATA']]*2

df_combined_100.loc[df_combined_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','DATA']] = \
df_combined_100.loc[df_combined_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','DATA']]*1.25

df_combined_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'DATA',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'TADIG_CODE_AL']].drop_duplicates().to_csv(join(load_folder,file_combined_100),index=False)

In [11]:
# Ищем в Allowed lists 'Partner_Combined' и 'P4_Combined' что нужно убрать/добавить

# Создаем файл

file_combined_al = 'allowed_list.csv'

df = df_combined_100[['NETWORK_ID','SPONSOR_ID']].drop_duplicates().copy()

df_al_=pd.read_csv(join(load_folder,file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al_.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al_ = df_al_[df_al_.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al_ = df_al_.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al_ = pd.merge(df_al_,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al_.drop('TADIG_CODE',inplace=True,axis=1)

pd.merge(df_al_,df,how='outer',on='NETWORK_ID').to_csv(join(load_folder,file_combined_al),index=False)

In [ ]:
# Подготовка файлов для модификации существующих Combine PP



In [67]:
df_s1_s2_sponsor.to_csv(join(load_folder,'result.csv'),index=False)